this series of tutorials can be found at:

https://www.analyticsvidhya.com/blog/2016/09/comprehensive-introduction-to-apache-spark-rdds-dataframes-using-pyspark/

https://www.analyticsvidhya.com/blog/2016/10/using-pyspark-to-perform-transformations-and-actions-on-rdd/

https://www.analyticsvidhya.com/blog/2016/10/spark-dataframe-and-operations/

to use jupyter notebook with pyspark: https://blog.sicara.com/get-started-pyspark-jupyter-guide-tutorial-ae2fe84f594f

In [1]:
import findspark
findspark.init()
import pyspark
import random

creating sparkcontext

In [2]:
from pyspark import SparkContext
sc = SparkContext()

In [3]:
sc.version

'2.2.0'

### To load a csv file

In [31]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
transactions = sqlContext.read.csv('../../../data/raw/transactions.csv',header=True)
user_logs = sqlContext.read.csv('../../../data/raw/user_logs.csv',header=True)
train = sqlContext.read.csv('../../../data/raw/train.csv',header=True)
test = sqlContext.read.csv('../../../data/raw/sample_submission_zero.csv',header=True)
members = sqlContext.read.csv('../../../data/raw/members.csv',header=True)

first 10 rows

problem is: the users in transactions and user logs are not unique.

That is, the same user may have multiple entries in these dataframes

number of unique users in transaction

In [17]:
# transactions.select('msno').distinct().count()

2363626

total number of entries in transaction

In [5]:
# transactions.count()

number of unique users in user_logs

In [19]:
# user_logs.select('msno').distinct().count()

5234111

total number of entries in user_logs

In [20]:
# user_logs.count()

392106543

rename the msno column in train to avoid duplicated colnames after join

In [34]:
transagg = sqlContext.read.csv('../../results/transagg.csv',header=True)

In [32]:
train = train.withColumnRenamed('msno', 'msno_1')

In [33]:
train1 = train.join(members, train['msno_1'] == members['msno'], 'left')

lazy evaluation, only calculates the dataframe when called

In [8]:
train1.count()

992931

In [35]:
train1 = train1.drop('msno')

to make sure we did the join correctly:

In [10]:
# train1.orderBy('msno_1').select('msno_1').distinct().head(10)

[Row(msno_1='+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o='),
 Row(msno_1='+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw='),
 Row(msno_1='+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc='),
 Row(msno_1='++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk='),
 Row(msno_1='++/UDNo9DLrxT8QVGiDi1OnWfczAdEwThaVyD0fXO50='),
 Row(msno_1='++/ZHqwUNa7U21Qz+zqteiXlZapxey86l6eEorrak/g='),
 Row(msno_1='++0/NopttBsaAn6qHZA2AWWrDg7Me7UOMs1vsyo4tSI='),
 Row(msno_1='++0BJXY8tpirgIhJR14LDM1pnaRosjD1mdO1mIKxlJA='),
 Row(msno_1='++0EzISdtKY48Z0GY62jer/LFQwrNIAbADdtU5xStGY='),
 Row(msno_1='++0wqjjQge1mBBe5r4ciHGKwtF/m322zkra7CK8I+Mw=')]

In [12]:
# train.orderBy('msno_1').head(10)

[Row(msno_1='+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=', is_churn='0'),
 Row(msno_1='+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=', is_churn='0'),
 Row(msno_1='+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=', is_churn='0'),
 Row(msno_1='++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=', is_churn='0'),
 Row(msno_1='++/UDNo9DLrxT8QVGiDi1OnWfczAdEwThaVyD0fXO50=', is_churn='0'),
 Row(msno_1='++/ZHqwUNa7U21Qz+zqteiXlZapxey86l6eEorrak/g=', is_churn='0'),
 Row(msno_1='++0/NopttBsaAn6qHZA2AWWrDg7Me7UOMs1vsyo4tSI=', is_churn='0'),
 Row(msno_1='++0BJXY8tpirgIhJR14LDM1pnaRosjD1mdO1mIKxlJA=', is_churn='0'),
 Row(msno_1='++0EzISdtKY48Z0GY62jer/LFQwrNIAbADdtU5xStGY=', is_churn='0'),
 Row(msno_1='++0wqjjQge1mBBe5r4ciHGKwtF/m322zkra7CK8I+Mw=', is_churn='0')]

Issue: when we have duplicates in the two dataframes we're trying to join, we end up with more rows in the resulting dataframe than both of its "parents".

To illustrate, if we have df1:

    Id | field_C | field_D
     A |   black | 11
     A |   white | 19
     A |  yellow | 20

and df2:

    Id | field_A | field_B 
     A |   cat   |  12     
     A |   dog   | 128     
     A |   dog   |  35     
 

when joining the two by Id, we will end up with

    Id | field_A | field_B | field_C | field_D
     A |   cat   |  12     |   black | 11
     A |   dog   | 128     |   black | 11
     A |   dog   |  35     |   black | 11
     A |   cat   |  12     |   white | 19
     A |   dog   |  128    |   white | 19
     A |   dog   |  35     |   white | 19
     A |   cat   |  12     |  yellow | 20
     A |   dog   |  128    |  yellow | 20
     A |   dog   |  35     |  yellow | 20
     
In general, you'll get $k * m$ rows with that the duplicated column value, where $k$ is the number of rows with that value in dataset 1 and $m$ is the number of rows with that value in dataset 2.

If we drop all duplicated msnos, we can avoid this issue but will suffer unaffordable information loss. If we don't drop any values, we will end up with a dataframe of amost 5 billion rows... I tried.

The duplicated msnos in **transactions** mostly originate from auto renewed records. The auto renewal in each month for the same user will be recorded as a separate entry, and the only differnce between these entries are the dates. We can probably aggregate this information into a new column like "number of autorenewals"? will talk with dear captain when get the chance.

The duplicated msnos in **user_logs** come from actual playing records for the same user on each day. These information are actually very valuable and shouldn't be neglected. We can potentially use a mean/median to capture characteristics for each user though.

Anyhow, we now have an effective way to manipulate even very large dataframes with pyspark. we can perform standard pandas operations such as filter, select, join, aggregate as well as map and reduce to them. the next step is to discuss how we want our data processed.

to just get rid of entries with duplicated msno when joining (don't want)

In [11]:
# train1 = train1.drop_duplicates(subset=['msno_1'])
# transactions_drop = transactions.drop_duplicates(subset=['msno'])
# train2 = train1.join(transactions_drop, train1['msno_1'] == transactions_drop['msno'], 'left')
# train2 = train2.drop('msno')
# train2 = train2.drop_duplicates(subset=['msno_1'])
# user_logs_drop = user_logs.drop_duplicates(subset=['msno'])
# train3 = train2.join(user_logs_drop, train2['msno_1'] == user_logs_drop['msno'], 'inner')

In [84]:
# train3.count()

869926

In [11]:
transactions.dtypes

[('msno', 'string'),
 ('payment_method_id', 'string'),
 ('payment_plan_days', 'string'),
 ('plan_list_price', 'string'),
 ('actual_amount_paid', 'string'),
 ('is_auto_renew', 'string'),
 ('transaction_date', 'string'),
 ('membership_expire_date', 'string'),
 ('is_cancel', 'string')]

In [13]:
# from datetime import datetime
# from pyspark.sql.functions import col, udf
# from pyspark.sql.types import DateType


In [14]:
# func =  udf (lambda x: datetime.strptime(x, '%m/%d/%Y'), DateType())

# transactions = transactions.withColumn('transaction_date', func(col('transaction_date')))
# transactions = transactions.withColumn('membership_expire_date', func(col('transaction_date')))

In [12]:
import numpy as np

In [44]:
# transactions.orderBy('msno').groupby('msno').agg({'payment_method_id':'mean'}).take(5)

[Row(msno='+++FOrTS7ab3tIgIh8eWwX4FqRv8w/FoiOuyXsFvphY=', avg(payment_method_id)=35.0),
 Row(msno='+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=', avg(payment_method_id)=38.0),
 Row(msno='+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=', avg(payment_method_id)=41.0),
 Row(msno='+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=', avg(payment_method_id)=39.0),
 Row(msno='+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=', avg(payment_method_id)=41.0)]

In [19]:
transagg = transactions.groupBy("msno").agg({"payment_method_id": "avg", "payment_plan_days": "avg", "plan_list_price": "avg", "actual_amount_paid": "avg", "is_auto_renew": 'min', "is_cancel" : "max"})

In [50]:
# transagg.toPandas().to_csv('../../results/transagg.csv')

In [13]:
user_logs.dtypes

[('msno', 'string'),
 ('date', 'string'),
 ('num_25', 'string'),
 ('num_50', 'string'),
 ('num_75', 'string'),
 ('num_985', 'string'),
 ('num_100', 'string'),
 ('num_unq', 'string'),
 ('total_secs', 'string')]

In [36]:
ulogsagg = user_logs.groupBy('msno').agg({"num_25":"avg","num_50":"avg","num_75":"avg",
                                          "num_985":"avg","num_100":"avg","num_unq":"avg",
                                          "total_secs":"avg"})

In [44]:
dat = train1.join(transagg, train1['msno_1'] == transagg['msno'], 'left')
dat = dat.drop('msno')
df = dat.join(ulogsagg, dat['msno_1'] == ulogsagg['msno'],'left')
df=df.drop('msno')
df = df.withColumnRenamed('msno_1', 'msno')
df.toPandas().to_csv('../../results/aggtrain.csv')

In [38]:
test = test.withColumnRenamed('msno', 'msno_1')
test1 = test.join(members, test['msno_1'] == members['msno'], 'left')
test1 = test1.drop('msno')

In [39]:
dat = test1.join(transagg, test1['msno_1'] == transagg['msno'], 'left')

In [40]:
dat = dat.drop('msno')

In [41]:
df = dat.join(ulogsagg, dat['msno_1'] == ulogsagg['msno'],'left')

In [42]:
df=df.drop('msno')
df = df.withColumnRenamed('msno_1','msno')

In [43]:
df.toPandas().to_csv('../../results/aggtest.csv')

# Ignore materials below $$$$$$$$$$$$$$$$$$$$$$$$$$$$$ 

### machine learning example

https://datahack.analyticsvidhya.com/contest/black-friday/

In [6]:
train = sqlContext.read.csv('../../../data/blackfriday/train.csv',header=True)
test = sqlContext.read.csv('../../../data/blackfriday/test.csv',header =True)

In [7]:
train.printSchema()

root
 |-- User_ID: string (nullable = true)
 |-- Product_ID: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Occupation: string (nullable = true)
 |-- City_Category: string (nullable = true)
 |-- Stay_In_Current_City_Years: string (nullable = true)
 |-- Marital_Status: string (nullable = true)
 |-- Product_Category_1: string (nullable = true)
 |-- Product_Category_2: string (nullable = true)
 |-- Product_Category_3: string (nullable = true)
 |-- Purchase: string (nullable = true)



In [8]:
train.head(10)

[Row(User_ID='1000001', Product_ID='P00069042', Gender='F', Age='0-17', Occupation='10', City_Category='A', Stay_In_Current_City_Years='2', Marital_Status='0', Product_Category_1='3', Product_Category_2=None, Product_Category_3=None, Purchase='8370'),
 Row(User_ID='1000001', Product_ID='P00248942', Gender='F', Age='0-17', Occupation='10', City_Category='A', Stay_In_Current_City_Years='2', Marital_Status='0', Product_Category_1='1', Product_Category_2='6', Product_Category_3='14', Purchase='15200'),
 Row(User_ID='1000001', Product_ID='P00087842', Gender='F', Age='0-17', Occupation='10', City_Category='A', Stay_In_Current_City_Years='2', Marital_Status='0', Product_Category_1='12', Product_Category_2=None, Product_Category_3=None, Purchase='1422'),
 Row(User_ID='1000001', Product_ID='P00085442', Gender='F', Age='0-17', Occupation='10', City_Category='A', Stay_In_Current_City_Years='2', Marital_Status='0', Product_Category_1='12', Product_Category_2='14', Product_Category_3=None, Purchase

number of rows

In [9]:
train.count()

550068

impute missing values

In [10]:
train.na.drop().count(),test.na.drop('any').count()

(166821, 71037)

In [11]:
train = train.fillna(-1)
test = test.fillna(-1)

summary

In [12]:
train.describe().show()

+-------+------------------+----------+------+------+-----------------+-------------+--------------------------+-------------------+------------------+------------------+------------------+-----------------+
|summary|           User_ID|Product_ID|Gender|   Age|       Occupation|City_Category|Stay_In_Current_City_Years|     Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|         Purchase|
+-------+------------------+----------+------+------+-----------------+-------------+--------------------------+-------------------+------------------+------------------+------------------+-----------------+
|  count|            550068|    550068|550068|550068|           550068|       550068|                    550068|             550068|            550068|            376430|            166821|           550068|
|   mean|1003028.8424013031|      null|  null|  null|8.076706879876669|         null|         1.468494139793958|0.40965298835780306| 5.404270017525106| 9.84232925112238

In [13]:
train.columns

['User_ID',
 'Product_ID',
 'Gender',
 'Age',
 'Occupation',
 'City_Category',
 'Stay_In_Current_City_Years',
 'Marital_Status',
 'Product_Category_1',
 'Product_Category_2',
 'Product_Category_3',
 'Purchase']

In [14]:
train.select('User_ID').show()

+-------+
|User_ID|
+-------+
|1000001|
|1000001|
|1000001|
|1000001|
|1000002|
|1000003|
|1000004|
|1000004|
|1000004|
|1000005|
|1000005|
|1000005|
|1000005|
|1000005|
|1000006|
|1000006|
|1000006|
|1000006|
|1000007|
|1000008|
+-------+
only showing top 20 rows



distribution of categorical features

In [15]:
train.select('Product_ID').distinct().count(), test.select('Product_ID').distinct().count()


(3631, 3491)

number of categories for Product_ID, which are in test but not in train by applying subtract method

In [16]:
diff_cat_in_train_test=test.select('Product_ID').subtract(train.select('Product_ID'))
diff_cat_in_train_test.distinct().count()# For distict count

46

Transforming categorical variables to labels

In [40]:
from pyspark.ml.feature import StringIndexer
plan_indexer = StringIndexer(inputCol = 'Product_ID', outputCol = 'product_ID_new')
labeller = plan_indexer.fit(train)

Above, we build a ‘labeller’ by applying fit() method on train Dataframe. Later we will use this ‘labeller’ to transform our train and test. Let us transform our train and test Dataframe with the help of labeller. We need to call transform method for doing that. We will store the transformation result in Train1 and Test1.



In [41]:
Train1 = labeller.transform(train)
Test1 = labeller.transform(test)

In [42]:
Train1.show()

+-------+----------+------+-----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+--------------+
|User_ID|Product_ID|Gender|  Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|Purchase|product_ID_new|
+-------+----------+------+-----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+--------------+
|1000001| P00069042|     F| 0-17|        10|            A|                         2|             0|                 3|              null|              null|    8370|         766.0|
|1000001| P00248942|     F| 0-17|        10|            A|                         2|             0|                 1|                 6|                14|   15200|         183.0|
|1000001| P00087842|     F| 0-17|        10|            A|                         2|     

successfully added one transformed column product_ID_new in our previous train Dataframe

### feature selection

In [43]:
df = sqlContext.createDataFrame([
    (1.0, 1.0, "a"),
    (0.0, 2.0, "b"),
    (0.0, 0.0, "a")
], ["y", "x", "s"])

In [51]:
rf = RFormula(formula="y ~ x + s")

In [52]:
rf.fit(df).transform(df).show()

+---+---+---+---------+-----+
|  y|  x|  s| features|label|
+---+---+---+---------+-----+
|1.0|1.0|  a|[1.0,1.0]|  1.0|
|0.0|2.0|  b|[2.0,0.0]|  0.0|
|0.0|0.0|  a|[0.0,1.0]|  0.0|
+---+---+---+---------+-----+



In [79]:
from pyspark.ml.feature import RFormula
formula = RFormula(formula="Purchase ~ Age + Occupation +City_Category + Stay_In_Current_City_Years + Product_Category_1 + Product_Category_2 + Gender",featuresCol="features",labelCol="label")

In [80]:
t1 = formula.fit(Train1)
train1 = t1.transform(Train1)
test1 = t1.transform(Test1)

In [81]:
train1.dtypes

[('User_ID', 'string'),
 ('Product_ID', 'string'),
 ('Gender', 'string'),
 ('Age', 'string'),
 ('Occupation', 'string'),
 ('City_Category', 'string'),
 ('Stay_In_Current_City_Years', 'string'),
 ('Marital_Status', 'string'),
 ('Product_Category_1', 'string'),
 ('Product_Category_2', 'string'),
 ('Product_Category_3', 'string'),
 ('Purchase', 'string'),
 ('product_ID_new', 'double'),
 ('features', 'vector'),
 ('label', 'double')]

something wrong with the version or whatever from here.. but dataframe transformation can be done with no problem

In [89]:
train1.select('label').show()
train1.select('features').show()

+-------+
|  label|
+-------+
|11529.0|
| 2312.0|
| 5917.0|
| 9163.0|
|  245.0|
| 2246.0|
| 5247.0|
| 3251.0|
| 3012.0|
|  774.0|
|  614.0|
| 5830.0|
| 1525.0|
| 2777.0|
|  830.0|
| 6344.0|
|11581.0|
| 1422.0|
| 3371.0|
| 4244.0|
+-------+
only showing top 20 rows



Py4JJavaError: An error occurred while calling o1365.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 211.0 failed 1 times, most recent failure: Lost task 0.0 in stage 211.0 (TID 1295, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$5: (string) => double)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIterator.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$8$$anon$1.hasNext(WholeStageCodegenExec.scala:395)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$2.apply(SparkPlan.scala:234)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$2.apply(SparkPlan.scala:228)
	at org.apache.spark.rdd.RDD$$anonfun$mapPartitionsInternal$1$$anonfun$apply$25.apply(RDD.scala:827)
	at org.apache.spark.rdd.RDD$$anonfun$mapPartition

### model: random forest

In [74]:
from pyspark.ml.regression import RandomForestRegressor
rf = RandomForestRegressor()

In [75]:
(train_cv, test_cv) = train1.randomSplit([0.7, 0.3])

In [76]:
model1 = rf.fit(train_cv)
predictions = model1.transform(test_cv)

Py4JJavaError: An error occurred while calling o891.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 183.0 failed 1 times, most recent failure: Lost task 0.0 in stage 183.0 (TID 1216, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$5: (string) => double)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIterator.sort_addToSorter$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIterator.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$8$$anon$1.hasNext(WholeStageCodegenExec.scala:395)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.c

In [101]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator()
mse = evaluator.evaluate(predictions,{evaluator.metricName:"mse" })
import numpy as np
np.sqrt(mse), mse

NameError: name 'predictions' is not defined

In [102]:
model = rf.fit(train1)
predictions1 = model.transform(test1)

Py4JJavaError: An error occurred while calling o1379.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 163.0 failed 1 times, most recent failure: Lost task 0.0 in stage 163.0 (TID 1656, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$5: (string) => double)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIterator.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$8$$anon$1.hasNext(WholeStageCodegenExec.scala:395)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.

In [103]:
df = predictions1.selectExpr("User_ID as User_ID", "Product_ID as Product_ID", 'prediction as Purchase')

NameError: name 'predictions1' is not defined

In [104]:
df.toPandas().to_csv('submission.csv')

KeyboardInterrupt: 